In [ ]:
# conexão com api do IBGE

import requests
url= "https://servicodados.ibge.gov.br/api/v1/localidades/estados"
resposta= requests.get(url)
print(resposta)

In [ ]:
dt=resposta.json()
dt



In [ ]:
# convertendo um arqv json para data freme sem trataçaõ

import requests
import pandas as pd

url="https://servicodados.ibge.gov.br/api/v1/localidades/estados"
resposta= requests.get(url)
df= resposta.json()
df= pd.DataFrame(df)
df


In [ ]:
# tratando o arquivo jason para normalizar no data freme
# jason_normalize transform achave em valores e colunas em linhas

import requests
import pandas as pd

url="https://servicodados.ibge.gov.br/api/v1/localidades/estados"
resposta= requests.get(url)
df= resposta.json()
df_normalize= pd.json_normalize(df)
df_normalize



In [ ]:
# LEVANDO API PARA O BANCO DE DADOS POSTGRES

In [ ]:
colunas=['id',
                'sigla', 
                'nome', 
                'regiao.id', 
                'regiao.sigla', 
                'regiao.nome']
df_normalize[colunas].rename(columns={'regiao.id':'regiao_id','regiao.sigla':'regiao_sigla','regiao.nome':'regiao_nome'},inplace=True)



In [ ]:
import psycopg2


dbname   = 'python1'
user     = 'postgres'
password = '99695421'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)

cursor=conn.cursor()

#deleta os dados antes de uma nova carga para nao acumular dados velhos
cursor.execute(" delete from public.teste10")

#carga de dados
for indice,colunas in df_normalize.iterrows():
    cursor.execute(""" INSERT INTO teste10(
                id,
                sigla, 
                nome 
                
            ) VALUES(%s,%s,%s)
                
            """ ,(
                colunas["id"],
                colunas["sigla"],
                colunas["nome"]
               
            )
                    
                    
                    
            )
    
conn.commit()
cursor.close()
conn.close()